In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

Для реализации функции rand_index была использована следующия структура данных:

Словарь, ключами которого являются ребра, а значениями - флаги 0/1 для случаев когда оба конца ребра находятся в одном и разных разбиениях графа соответсвенно.

Изначальное разбиение получалось следующим образом:
    
  - 0...(N/2-1) вершины принадлежат разбиению 1;
  - N/2...N принадлежат разбиению 2.

Это обусловлено работой генератора графа з библиотеки NetworkX.

С ним сравнивалось разбиение на кластеры, полученное алгоритмом Кернигана-Лина, также реализованного в данной библиотеке.

In [2]:
def graph_to_edge_structure(graph, partition):
    (Cluster1,Cluster2) = partition
    Edges = [(X,Y) for (X,Y,_) in list(nx.to_edgelist(graph))]
    Structure = dict()
    for (X,Y) in Edges:
        if (X in Cluster1 and Y in Cluster1) or (X in Cluster2 and Y in Cluster2):
            Structure[(X,Y)] = 0;
        else:
            Structure[(X,Y)] = 1
    return Structure

Для обоих разбиений - до и после кластеризации строилась указанная выше структура и после этого словари построчно сравнивались.

Выделялись следующие случаи:

  - Группа А. В обеих структурах значения совпадали и были равны 0. Это означает, что в обоих разбиениях обе вершины ребра находились в одном кластере.
  - Группа В. В обеих структурах значения совпадали и были равны 1. Это означает, что в обоих разбиениях вершины ребра находились в разных кластерах.
  - Группа С. Значение в первой структуре было меньше значения во второй. Это означает, что до кластеризации обе вершины были в одном кластере, а после оказались в разных.
  - Группа D. Значение в первой структуре было больше значения во второй. Это означает, что до кластеризации вершины были в в разных кластерах, а после оказались в одном.

На основе количества элементов, попавших в разные группы, значение Rand Index считалось по следующей формуле:
    
    RandIndex = [size(A) + size(B)] / [size(A) + size(B) + size(C) + size(D)]

In [3]:
def rand_index(graph1, partition):#partiton = (Cluster1,Cluster2) after 
    Structure1 = graph_to_edge_structure(graph1,([i for i in range(0,3)],[i for i in range(3,6)]))
    Structure2 = graph_to_edge_structure(graph,partition)
    classes=[0,0,0,0]
    
    print("Partition: {}\n".format(partition))
    for row in Structure1:
        if Structure1[row] == Structure2[row] == 0:
            classes[0] += 1;
        elif Structure1[row] == Structure2[row] == 1:
            classes[1] += 1;
        elif Structure1[row] < Structure2[row]:
            classes[2] += 1;
        else:
            classes[3] += 1
    
    print("Classes: {}\n".format(classes))
    assert np.sum(classes) == len(list(nx.to_edgelist(graph))), "Wrong number!"
    
    r_index = (classes[0] + classes[1])/np.sum(classes)
    return r_index

In [ ]:
X=49
graph = nx.planted_partition_graph(2,50, X/49, 0/49)
plt.figure(figsize=(10,10))
nx.draw_shell(graph, with_labels=True)